<h1>
<hr style=" border:none; height:3px;">
<center>Evaluation pipeline</center>
<hr style=" border:none; height:3px;">
</h1>

<center><img src='https://netacad.centralesupelec.fr/img/cs.jpg' width=200></center>

<h4><center>Louis LHOTTE | Clément VERON | Edouard SEGUIER</center></h4>

In [1]:
# Imports
import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

c:\Users\Louis\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# I - Evaluation

In [2]:
def evaluate_images_with_clip(image_1_path, image_2_path, prompt):
    """
    Evaluate two images based on their relevance to a given text prompt using the CLIP metric.

    Args:
        image_1_path (str): Path to the first image.
        image_2_path (str): Path to the second image.
        prompt (str): The text prompt for evaluation.

    Returns:
        dict: A dictionary containing similarity scores for each image.
    """
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

    image_1 = Image.open(image_1_path).convert("RGB")
    image_2 = Image.open(image_2_path).convert("RGB")
    inputs = processor(text=[prompt], images=[image_1, image_2], return_tensors="pt", padding=True)

    outputs = model(**inputs)
    image_embeddings = outputs.image_embeds
    text_embeddings = outputs.text_embeds

    similarity = torch.nn.functional.cosine_similarity(image_embeddings, text_embeddings)

    result = {
        "image_1_score": similarity[0].item(),
        "image_2_score": similarity[1].item(),
    }

    return result

In [9]:
if __name__ == "__main__":
    image_1_path = "input.jpg"
    image_2_path = "output.png"
    prompt = "Transform this image into a clean, sharp, modern, bold 2D logo design for a mountaineering company. Integrate sharp and minimalistic design elements, the whole image should be transformed into a traditional logo"

    scores = evaluate_images_with_clip(image_1_path, image_2_path, prompt)
    print("Input image Score:", scores["image_1_score"])
    print("Ouput image Score:", scores["image_2_score"])
    print("Delta Image:", scores["image_2_score"] - scores["image_1_score"])

Input image Score: 0.2334834784269333
Ouput image Score: 0.2928134500980377
Delta Image: 0.05932997167110443
